In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import mysql.connector
import pandas as pd
import numpy as np

import mysql.connector
import pandas as pd

# MySQL 연결 설정
cnx = mysql.connector.connect(
    host='',
    user='',
    password='',
    database='ball_movement'
)

# 쿼리 생성
query_ball = "SELECT * FROM ball_df"
query_sequence_df = "SELECT * FROM sequence_df"
query_events_df = "SELECT * FROM events_df"

# DataFrame으로 읽기
ball_df = pd.read_sql(query_ball, con=cnx)
sequence_df = pd.read_sql(query_sequence_df, con=cnx)
events_df = pd.read_sql(query_events_df, con=cnx)

# 연결 종료 (선택적)
cnx.close()

##### net 시점에서의 공의 높이를 구합니다.

공의 높이만 구하는 문제라면 x축은 고려하지 않아도 되므로 카메라 각도상 찌그러진 탁구대 마름모의 높이를 탁구대의 세로로 볼 수 있습니다.

이렇게 기존 카메라상의 탁구대의 가로/세로 비율과 기존 탁구대의 가로/세로 비율을 비교하여 카메라의 각도를 추정합니다.\
\
이렇게 추정된 카메라의 각도를 이용하여 공의 높이를 구합니다.

하지만 이는 여전히 근사치로 추정한 값이고 정확한 3d 변환을 통해 공의 높이를 구하려면 다른 카메라 각도에서의 추가적인 Dataset이 필요합니다. 해당 결과로는 두 플레이어 간의 높이 비교 정도로만 사용할 수 있을 것 같습니다.

In [3]:
import pandas as pd
import numpy as np

# 카메라 시점 탁구대 꼭지점 좌표 설정
table_corners = {
    'left_bottom': (305, 103),
    'left_top': (510, 217),
    'right_top': (1297, 202),
    'right_bottom': (1435, 76)
}

# 카메라 시점의 탁구대 크기 계산 (카메라 각도 추정시 사용)

# 가로 길이 계산: 왼쪽 위와 오른쪽 위 꼭지점 사이의 거리를 유클리드 거리로 계산합니다.
image_table_width = np.sqrt((table_corners['right_top'][0] - table_corners['left_top'][0])**2 +
                            (table_corners['right_top'][1] - table_corners['left_top'][1])**2)
# 세로 길이 계사니 왼쪽 위와 왼쪽 아래 꼭지점 사이의 거리를 유클리드 거리로 계산합니다. 
image_table_length = np.sqrt((table_corners['left_top'][0] - table_corners['left_bottom'][0])**2 +
                             (table_corners['left_top'][1] - table_corners['left_bottom'][1])**2)

# 탁구대의 표준 크기와 이미지 상의 크기를 비교하여 카메라의 각도를 추정 (얼마나 테이블이 찌그러져 있는지)
standard_ratio = 2.74 / 1.525  # 탁구대 표준 가로 세로 비율
image_ratio = image_table_width / image_table_length
camera_angle = standard_ratio / image_ratio

# net 이벤트만 필터링
events_df_net = events_df[events_df['event'] == 'net'].copy()

# 카메라 각도 보정 및 높이 계산
# 카메라 각도를 보정하여 y좌표를 수정하고, 이를 바탕으로 탁구대 표면으로부터의 실제 높이를 계산합니다.
net_height = 0.9125  # 탁구대 + 네트 실제 높이 (미터)
height_ratio = (max(table_corners['left_top'][1], table_corners['right_top'][1]) - 
                min(table_corners['left_bottom'][1], table_corners['right_bottom'][1])) / net_height
events_df_net['revised_y'] = events_df_net['y'] * camera_angle
events_df_net['height'] = events_df_net['revised_y'].apply(lambda y: (y / height_ratio))

# 탁구대까지 높이 빼주기
events_df_net['height'] = events_df_net['height'] - 0.76 + 0.1 # 높이 보정
events_df_net['height'] = events_df_net['height'].round(2)

events_df_net = events_df_net.drop(columns= 'revised_y')
events_df_net

,index,frame,x,y,event,player,height
1,1,33,897,228,net,player_A,0.13
4,4,2211,889,240,net,player_B,0.17
5,5,2270,902,251,net,player_A,0.21
8,8,9267,899,238,net,player_B,0.16
10,10,9343,905,244,net,player_A,0.19
...,...,...,...,...,...,...,...
325,367,77955,906,267,net,player_A,0.27
327,369,79091,893,245,net,player_A,0.19
329,371,79152,895,263,net,player_B,0.25
331,373,79225,893,239,net,player_A,0.17


In [8]:
events_df_net['height'].mean()

0.7128604108108173

In [10]:
# mysql로 보내기

from sqlalchemy import create_engine
 
db_connection_path = ''
db_connection = create_engine(db_connection_path)

conn = db_connection.connect()

events_df_net.to_sql('events_df_net', con = db_connection, if_exists= 'replace')

162